In [1]:
import numpy as np
from MetaLearner import *
import pickle

In [ ]:
# todo: https://stackoverflow.com/questions/24168569/how-to-add-names-to-a-numpy-array-without-changing-its-dimension

In [ ]:
# This file runs the experiment

In [2]:
file_name = "/Users/arberimbibaj/Documents/Master Thesis ETH/DataSets /Generated/data_synthetic.pkl"
open_file = open(file_name, "rb")
data = pickle.load(open_file)
open_file.close()

In [3]:
n_setups = 2
sample_sizes = [200, 200]
n_runs = 2
learners = [TLearner('rf'), SLearner('rf'), XLearner('rf'), RLearner('rf'), DRLearner('rf'), RALearner('rf'),
            PWLearner('rf'), ULearner('rf'),
            TLearner('lasso'), SLearner('lasso'), XLearner('lasso'), RLearner('lasso'), DRLearner('lasso'),
            RALearner('lasso'), PWLearner('lasso'), ULearner('lasso'),
            TLearner('nn'), SLearner('nn'), XLearner('nn'), RLearner('nn'), DRLearner('nn'), RALearner('nn'),
            PWLearner('nn'), ULearner('nn')]


In [ ]:
# What we want: For each setting and base learner --> matrix which includes the MSE, columns: Metalearners + SampleSize, rows: Run(1-10)
# --> So we can plot the mean and 1se of the MSE for each Metalearer, each samplesize; giving the setting and baselearner
# --> number of plots: settings*baselearner.

In [4]:
results = []

for i in range(n_setups):
    results.append([])

for i in range(n_setups):
    for baselearner in range(3):
        results[i].append([])

results

[[[], [], []], [[], [], []]]

In [5]:
dim = data[0][0][0][0].shape[1]  # dimension of x

# helper function to get y, x, w, tau, out of data[][][][]
def get_variables(dataset, setup, samplesize, run, train_test):
    y = dataset[setup][samplesize][run][train_test][:, 0]
    x = dataset[setup][samplesize][run][train_test][:, 1:(dim-2)]
    w = dataset[setup][samplesize][run][train_test][:, (dim-2)]
    tau = dataset[setup][samplesize][run][train_test][:, (dim-1)]
    return y, x, w, tau

In [6]:
b = 0
m = 0
s = 0

for i in range(n_setups):
    # array for all mses for one setup
    setup_mse = np.empty(shape=(0, 24))
    s = 0  # restart index for samplesize
    for size in sample_sizes:
        # array for all mses for one setup and samplesize.
        size_mse = np.empty(shape=(0, 24))
        for r in range(n_runs):
            # array for all mses in one setup, samplesize and run.
            mses = np.empty(shape=(1, 24))
            # get data for specific setup, samplesize and run.
            temp_y_train, temp_x_train, temp_w_train, temp_tau_train = get_variables(dataset=data, setup=i,
                                                                                     samplesize=s, run=r,
                                                                                     train_test=0)
            temp_y_test, temp_x_test, temp_w_test, temp_tau_test = get_variables(dataset=data, setup=i,
                                                                                 samplesize=s, run=r,
                                                                                 train_test=1)
            # restart index for metalearner
            m = 0
            for l in learners:
                # training and testing MetaLearner.
                learner = l
                learner.fit(temp_x_train, temp_y_train, temp_w_train)
                predictions = learner.predict(temp_x_test)
                temp_mse = ((predictions - temp_tau_test) ** 2).mean()
                # append mse of specific metalearner to 'mses'.
                mses[0, m] = temp_mse
                # update index
                m += 1
            # append 'mses' to 'size_mse'.
            size_mse = np.append(size_mse, mses, axis=0)
        # append 'size_mse' to 'setup_mse'.
        setup_mse = np.append(setup_mse, size_mse, axis=0)
        # update index
        s += 1
    # append to results
    results[i][0] = setup_mse[:, 0:8]
    results[i][1] = setup_mse[:, 8:16]
    results[i][2] = setup_mse[:, 16:24]





# this should make sense i guess! :)
# TODO: CHECK IF IT MAKES IT FASTER IF EVERYTHING IS EITHER IN NP OR IN LIST/ARRAY BASE PYTHON.
# TODO: WHILE TRAINING: ConvergenceWarning: Try increasing tol. DONE: :)
# TODO: solve : "Input y contains infinity or a value too large for dtype('float64')."
# TODO: tensorflow warning 5: https://github.com/tensorflow/tensorflow/issues/34025 AND https://stackoverflow.com/questions/58972225/tensorflow2-warning-using-tffunction.

### IT TOOK 5M 40S FOR --> SAMPLE SIZES [200, 200], TEST SIZE = 50, 2 SETUPS, 2 RUNS LOL

Fitting RF for mu_0
Fitting RF for mu_1
Fitting RF for mu_x
Fitting RF for mu_0
Fitting RF for mu_1
Fitting RF for tau_0
Fitting RF for tau_1
Fitting RF for e_x
Fitting RF for mu_x
Fitting RF for e_x
Fitting RF for tau_x
Fitting RF for mu_0
Fitting RF for mu_1
Fitting RF for e_x
Fitting RF for tau_x
Fitting RF for mu_0
Fitting RF for mu_1
Fitting RF for tau_x
Fitting RF for e_x
Fitting RF for tau_x
Fitting RF for mu_x
Fitting RF for e_x
Fitting RF for tau_x
Fitting Lasso for mu_0
Fitting Lasso for mu_1
Fitting Lasso for mu_x
Fitting Lasso for mu_0
Fitting Lasso for mu_1
Fitting Lasso for tau_0
Fitting Lasso for tau_1
Fitting Lasso for e_x
Fitting Lasso for mu_x
Fitting Lasso for e_x
Fitting Lasso for tau_x
Fitting lasso for mu_0
Fitting lasso for mu_1
Fitting lasso for e_x
Fitting lasso for tau_x
Fitting Lasso for mu_0
Fitting Lasso for mu_1
Fitting Lasso for tau_x
Fitting Lasso for e_x
Fitting Lasso for tau_x
Fitting Lasso for mu_x
Fitting Lasso for e_x
Fitting Lasso for tau_x
Trainin

In [15]:
results[0][2]  # NICE :)

array([[2.63915079e+00, 2.84076208e-01, 2.01535158e+00, 9.32191106e-01,
        3.64672269e+00, 2.97932948e+00, 9.61618524e+01, 8.84802055e-01],
       [6.36597020e+00, 4.41640701e-01, 2.31011468e+00, 4.23029349e+00,
        3.16558000e+00, 5.12849656e+00, 1.50659185e+02, 5.18250425e+00],
       [1.82256991e+00, 1.79153159e-01, 6.91466612e-01, 7.24130093e+00,
        5.77499473e-01, 1.75643935e+00, 6.84936020e+01, 6.89073489e+00],
       [3.78212105e+00, 4.17684090e-01, 1.66642819e+00, 6.04114946e+00,
        9.05181505e-01, 4.18108313e+00, 2.17496751e+02, 4.71001566e+00]])

small check

In [16]:
mses

array([[5.39904726e+00, 1.62035526e-02, 3.41632611e+00, 8.53723210e+00,
        4.18814312e+00, 3.62821884e+00, 4.86883242e+00, 1.03440314e+01,
        2.33151928e+00, 2.30794607e-01, 7.89986783e-01, 4.76328607e-01,
        7.46131186e-01, 8.60046992e-01, 2.97617878e-01, 5.05113092e-01,
        2.43346444e+01, 2.35734119e+00, 1.21145239e+01, 1.08708231e+01,
        2.73133027e+00, 2.63222003e+01, 2.78196544e+01, 1.09634150e+01]])

In [17]:
size_mse

array([[8.35256837e+00, 1.90981560e-03, 4.14908013e+00, 4.02424087e+01,
        8.71280249e+00, 8.18541222e+00, 1.13962376e+01, 8.07310756e+01,
        5.38428691e+01, 5.90157811e-01, 1.81801267e+01, 1.53441568e+00,
        3.95228809e+00, 2.14032564e+01, 1.97406846e+01, 1.81519375e+00,
        1.91514118e+01, 1.86265939e+00, 6.86527103e+00, 5.22751361e+00,
        4.27699980e+00, 1.45954538e+01, 8.09282684e+01, 7.61127490e+00],
       [5.39904726e+00, 1.62035526e-02, 3.41632611e+00, 8.53723210e+00,
        4.18814312e+00, 3.62821884e+00, 4.86883242e+00, 1.03440314e+01,
        2.33151928e+00, 2.30794607e-01, 7.89986783e-01, 4.76328607e-01,
        7.46131186e-01, 8.60046992e-01, 2.97617878e-01, 5.05113092e-01,
        2.43346444e+01, 2.35734119e+00, 1.21145239e+01, 1.08708231e+01,
        2.73133027e+00, 2.63222003e+01, 2.78196544e+01, 1.09634150e+01]])

In [18]:
setup_mse

array([[1.96537533e+00, 3.35597974e-03, 9.30893641e-01, 4.27387465e+00,
        1.90672245e+00, 1.92493198e+00, 3.49678582e+00, 4.76913753e+00,
        1.11283226e+00, 0.00000000e+00, 2.73109759e-01, 1.54595104e-01,
        5.41503847e-01, 1.04645801e+00, 2.04329133e+00, 1.58757720e-01,
        6.93970144e+00, 6.60071572e-01, 3.96701531e+00, 7.82208146e+00,
        5.40187407e-01, 5.09941716e+00, 3.59610625e+00, 1.43743670e+01],
       [7.16293784e+00, 2.31834380e-02, 1.43780381e+00, 5.11771216e+01,
        3.31696077e+00, 3.39018449e+00, 2.15809912e+01, 6.48365038e+01,
        2.67661472e+01, 1.62039039e-01, 1.13515856e+01, 2.66536010e-01,
        1.11015144e+00, 1.97699241e+01, 4.15759332e+01, 2.66514897e-01,
        9.82032908e+00, 7.82814731e-01, 5.16740544e+00, 7.23274237e+00,
        3.58107209e+00, 9.49107634e+00, 4.33480003e+01, 8.78749192e+00],
       [8.35256837e+00, 1.90981560e-03, 4.14908013e+00, 4.02424087e+01,
        8.71280249e+00, 8.18541222e+00, 1.13962376e+01, 8.0731

In [ ]:
# NICE :)))